In [ ]:
#default_exp models.metrics

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from grade_classif.imports import *
from torchmetrics import Metric
from kornia.color import rgb_to_grayscale

In [ ]:
# export
def _input_format_classification(
    preds: torch.Tensor, target: torch.Tensor, threshold: float = 0.5
) -> Tuple[torch.Tensor, torch.Tensor]:
    """Convert preds and target tensors into label tensors
    Args:
        preds: either tensor with labels, tensor with probabilities/logits or
            multilabel tensor
        target: tensor with ground true labels
        threshold: float used for thresholding multilabel input
    Returns:
        preds: tensor with labels
        target: tensor with labels
    """
    if not (preds.ndim == target.ndim or preds.ndim == target.ndim + 1):
        raise ValueError("preds and target must have same number of dimensions, or one additional dimension for preds")

    if preds.ndim == target.ndim + 1:
        # multi class probabilites
        preds = torch.argmax(preds, dim=1)

    if preds.ndim == target.ndim and preds.is_floating_point():
        # binary or multilabel probablities
        preds = (preds >= threshold).long()
    return preds, target

In [ ]:
# export
def _reshape(input, target):
    n = target.shape[0]
    input = torch.softmax(input, dim=1)
    input = input.argmax(dim=-1).view(n,-1)
    target = target.view(n,-1)
    return input, target

In [ ]:
# export
def accuracy(tp, fp, tn, fn):
    return (tp+tn)/(tp+fp+fn+tn+1e-7)

In [ ]:
# export
def fp_rate(tp, fp, tn, fn):
    return fp/(fp+tn+1e-7)

In [ ]:
# export
def fn_rate(tp, fp, tn, fn):
    return fn/(fn+tp+1e-7)

In [ ]:
# export
def precision(tp, fp, tn, fn, cat=1):
    if cat == 1:
        return tp/(fp+tp+1e-7)
    else:
        return tn/(fn+tn+1e-7)

In [ ]:
# export
def recall(tp, fp, tn, fn, cat=1):
    if cat == 1:
        return tp/(fn+tp+1e-7)
    else:
        return tn/(fp+tn+1e-7)

In [ ]:
# export
def f_beta(tp, fp, tn, fn, beta=1, cat=1):
    prec = precision(tp, fp, tn, fn, cat)
    rec = recall(tp, fp, tn, fn, cat)
    return ((1+beta**2)*prec*rec+1e-7)/(beta**2*prec+rec+1e-7)

In [ ]:
# export
def f_1(tp, fp, tn, fn, cat=1):
    return f_beta(tp, fp, tn, fn, beta=1, cat=cat)

In [ ]:
# export
def pcc(mu_x, sigma_x, mu_y, sigma_y, mu_xy):
    return (mu_xy-mu_x*mu_y)/(sigma_x*sigma_y+1e-7)

In [ ]:
print(" ".join("""
numpy>=1.18
torch>=1.7
torchvision>=0.8
pytorch-lightning>=1.2
torchmetrics>=0.2
opencv-python>=4.1
timm>=0.3
tqdm>=4.35
pandas>=1.2
openslide-python>=1.2
staintools>=2.1
fastcore>=1.3
matplotlib>=3.3
scikit-image>=0.17
pillow>=7.2
albumentations>=0.4
kornia>=0.5
scipy>=1.6
""".split("\n")))

 numpy>=1.18 torch>=1.7 torchvision>=0.8 pytorch-lightning>=1.2 torchmetrics>=0.2 opencv-python>=4.1 timm>=0.3 tqdm>=4.35 pandas>=1.2 openslide-python>=1.2 staintools>=2.1 fastcore>=1.3 matplotlib>=3.3 scikit-image>=0.17 pillow>=7.2 albumentations>=0.4 kornia>=0.5 scipy>=1.6 


In [ ]:
# export
def ssim(mu_x, sigma_x, mu_y, sigma_y, mu_xy, k1=0.01, k2=0.01):
    return (
        (2 * mu_x * mu_y + k1)
        * (2 * (mu_xy - mu_x * mu_y) + k2)
        / ((mu_x ** 2 + mu_y ** 2 + k1) * (sigma_x ** 2 + sigma_y ** 2 + k2))
    )

In [ ]:
# export
def Accuracy(Metric):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.add_state("true", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")
        
    def update(self, preds, target):
        preds, target = _input_format_classification(preds, target, self.threshold)
        self.true += (preds == target).sum()
        self.total += target.numel()
    
    def compute(self):
        return self.true.float() / self.total

In [ ]:
# export
class ClassifMetrics(Metric):
    def __init__(self, n_classes=2, threshold=0.5, **kwargs):
        super().__init__(**kwargs)
        self.n_classes = n_classes
        self.threshold = threshold
        self.add_state("tp", default=torch.zeros(n_classes), dist_reduce_fx="sum")
        self.add_state("tn", default=torch.zeros(n_classes), dist_reduce_fx="sum")
        self.add_state("fp", default=torch.zeros(n_classes), dist_reduce_fx="sum")
        self.add_state("fn", default=torch.zeros(n_classes), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds, target):
        preds, target = _input_format_classification(preds, target, self.threshold)
        for k in range(self.n_classes):
            self.tp[k] += ((preds == k) & (target == k)).sum()
            self.tn[k] += ((preds != k) & (target != k)).sum()
            self.fp[k] += ((preds == k) & (target != k)).sum()
            self.fn[k] += ((preds != k) & (target == k)).sum()
        self.total += target.numel()

    def compute(self):
        precision = self.tp.float() / (self.tp + self.fp)
        recall = self.tp.float() / (self.tp + self.fn)
        ret = {
            "precision": precision,
            "recall": recall,
            "f1": (2 * precision * recall) / (precision + recall),
            "accuracy": self.tp.sum().float() / self.total
        }
        return ret

In [ ]:
# export
class NormMetrics(Metric):
    def __init__(self, k1=0.01, k2=0.01, **kwargs):
        super().__init__(**kwargs)
        self.add_state("mu_x", default=[], dist_reduce_fx="cat")
        self.add_state("mu_y", default=[], dist_reduce_fx="cat")
        self.add_state("mu_xy", default=[], dist_reduce_fx="cat")
        self.add_state("sigma_x", default=[], dist_reduce_fx="cat")
        self.add_state("sigma_y", default=[], dist_reduce_fx="cat")
        
    def update(self, preds, targets):
        y = rgb_to_grayscale(targets.detach()).squeeze(1)
        y_hat = rgb_to_grayscale(preds.detach()).squeeze(1)
        self.mu_x.append(torch.mean(y, axis=(-2, -1)).float())
        self.sigma_x.append(torch.std(y, axis=(-2, -1)).float())
        self.mu_y.append(torch.mean(y_hat, axis=(-2, -1)).float())
        self.sigma_y.append(torch.std(y_hat, axis=(-2, -1)).float())
        self.mu_xy.append(torch.mean(y * y_hat, axis=(-2, -1)).float())
    
    def compute(self):
        mu_x = torch.cat(self.mu_x)
        sigma_x = torch.cat(self.sigma_x)
        mu_y = torch.cat(self.mu_y)
        sigma_y = torch.cat(self.sigma_y)
        mu_xy = torch.cat(self.mu_xy)
        ret = {}
        ret["ssim"] = ssim(mu_x, sigma_x, mu_y, sigma_y, mu_xy).mean()
        ret["pcc"] = pcc(mu_x, sigma_x, mu_y, sigma_y, mu_xy).mean()
        ret["cd"] = (sigma_y / mu_y - sigma_x / mu_x).mean()
        return ret

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data.read.ipynb.
Converted 11_data.loaders.ipynb.
Converted 12_data.dataset.ipynb.
Converted 13_data.utils.ipynb.
Converted 14_data.transforms.ipynb.
Converted 15_data.color.ipynb.
Converted 16_data.modules.ipynb.
Converted 20_models.plmodules.ipynb.
Converted 21_models.modules.ipynb.
Converted 22_models.utils.ipynb.
Converted 23_models.hooks.ipynb.
Converted 24_models.metrics.ipynb.
Converted 25_models.losses.ipynb.
Converted 80_params.defaults.ipynb.
Converted 81_params.parser.ipynb.
Converted 99_index.ipynb.
